# Try to Recreate the Sound Module

## Prep - Get our Build Tools

We have two tools to allow us to build and link ASM68K source files the way it would have been done back in the day. The first is 'rmac', a successor to the popular `madmac` assembler used by Minter:

In [3]:
import os
!rm -rf rmac
!git clone https://github.com/mwenge/rmac.git
os.chdir('rmac')
!make -j5
os.chdir('..')

Cloning into 'rmac'...
remote: Enumerating objects: 2050, done.
remote: Counting objects: 100% (2050/2050), done.
remote: Compressing objects: 100% (390/390), done.
remote: Total 2050 (delta 1641), reused 2050 (delta 1641), pack-reused 0
Receiving objects: 100% (2050/2050), 740.86 KiB | 2.58 MiB/s, done.
Resolving deltas: 100% (1641/1641), done.
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c kwgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c 68kgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c debug.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c dsp56k.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c dsp56kgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c eagen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -o dsp56kgen dsp56kgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -o 68kgen 68kgen.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -o kwgen kwgen.c
gcc -std=c99 -D

In file included from macro.h:12,
                 from macro.c:9:
macro.c: In function ‘ExitMacro’:
rmac.h:78:30: note: ‘#pragma message: !!! Bad macro exiting !!!’
   78 |         #define DO_PRAGMA(x) _Pragma (#x)
      |                              ^~~~~~~
rmac.h:79:31: note: in expansion of macro ‘DO_PRAGMA’
   79 |         #define WARNING(desc) DO_PRAGMA(message (#desc))
      |                               ^~~~~~~~~
macro.c:54:1: note: in expansion of macro ‘WARNING’
   54 | WARNING(!!! Bad macro exiting !!!)
      | ^~~~~~~
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c object.c
./dsp56kgen dsp56k.tab <dsp56k.mch >dsp56ktab.h
./kwgen mr <risc.tab >risckw.h
./kwgen mp <6502.tab >6502kw.h
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c rmac.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c sect.c
gcc -std=c99 -D_DEFAULT_SOURCE -g -D__GCCUNIX__ -I. -O2 -c symbol.c
./kwgen kw <kw.tab >kwtab.h
./68kgen 68k.tab <68k.mch >68ktab.h
gcc -std=c99 -D_DEF

Next we build `rln`, the successor to `aln` the linker-de-jour for Jaguar:

In [4]:
import os
!rm -rf rln
!git clone http://shamusworld.gotdns.org/git/rln
os.chdir('rln')
!make -j5
os.chdir('..')

Cloning into 'rln'...
remote: Enumerating objects: 216, done.
remote: Counting objects: 100% (216/216), done.
remote: Compressing objects: 100% (216/216), done.
remote: Total 216 (delta 106), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (216/216), 184.15 KiB | 255.00 KiB/s, done.
Resolving deltas: 100% (106/106), done.
/bin/rm -f rln.o
gcc -g -I. -D__GCCUNIX__ -O2 -Wno-format -c rln.c
gcc -g -I. -D__GCCUNIX__ -O2 -Wno-format -o rln rln.o 


## Try recreating using a single source file

In [145]:
#!./rmac/rmac -fa -i moomoo_header.s -o moomoo_header.o
#!./rmac/rmac -s -fb -i moomoo.s -o moomoo.o
!./rmac/rmac -s ~oall -n -fb -l*moomooprn -i moomoo.s -o moomoo.o

Legacy mode OFF
moomoo.s 90: Warning: unoptimized short branch
moomoo.s 1026: Warning: unoptimized short branch


In [146]:
!./rln/rln -n -z -u -a 4040 xd xd -e moomoo.o -o moomoo_src.abs

In [147]:
!diff -s moomoo_src.abs moomoo.bin

Files moomoo_src.abs and moomoo.bin are identical


In [148]:
!diff -y -W 120 <(dd status=none if=moomoo_src.abs|xxd) \
    <(dd status=none if=moomoo.bin|xxd)


00000000: 4ef9 0000 41ba 4ef9 0000 425e 4ef9 0000  N...A	00000000: 4ef9 0000 41ba 4ef9 0000 425e 4ef9 0000  N...A
00000010: 41d6 4ef9 0000 45e2 4ef9 0000 4606 4ef9  A.N..	00000010: 41d6 4ef9 0000 45e2 4ef9 0000 4606 4ef9  A.N..
00000020: 0000 4370 4ef9 0000 44ea 4ef9 0000 4546  ..CpN	00000020: 0000 4370 4ef9 0000 44ea 4ef9 0000 4546  ..CpN
00000030: 4ef9 0000 4598 4ef9 0000 45b2 4ef9 0000  N...E	00000030: 4ef9 0000 4598 4ef9 0000 45b2 4ef9 0000  N...E
00000040: 45ca 4ef9 0000 464e 4ef9 0000 4660 4ef9  E.N..	00000040: 45ca 4ef9 0000 464e 4ef9 0000 4660 4ef9  E.N..
00000050: 0000 4660 4ef9 0000 4660 4ef9 0000 4470  ..F`N	00000050: 0000 4660 4ef9 0000 4660 4ef9 0000 4470  ..F`N
00000060: 0000 4e75 3f00 08f9 0003 0000 40a0 4240  ..Nu?	00000060: 0000 4e75 3f00 08f9 0003 0000 40a0 4240  ..Nu?
00000070: 1039 0000 40a0 33c0 00f0 00e0 301f 4e75  .9..@	00000070: 1039 0000 40a0 33c0 00f0 00e0 301f 4e75  .9..@
00000080: 436f 7079 7269 6768 7420 2831 3939 3329  Copyr	00000080: 436f 7079 7269 6768 7